In [1]:
import os
import sys
from dotenv import load_dotenv

from helpers.helper_functions import *
from helpers.evalute_rag import *

from langchain_experimental.text_splitter import SemanticChunker
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

/tmp/ipykernel_13579/3395977847.py:5: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from helpers.helper_functions import *
/home/stackops/.local/lib/python3.10/site-packages/deepeval/__init__.py:53: UserWarning: You are using deepeval version 2.3.7, however version 2.4.1 is available. You should consider upgrading via the "pip install --upgrade deepeval" command.
  warnings.warn(


In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = (
    "/home/stackops/secret/work/vngcloud/ai-platform/vertex-ai-credential.json"
)


## Define file path


In [3]:
path = "/home/stackops/langchain-labs/data/vks/pdf/vi/02-nlb.pdf"

## Read PDF to string


In [4]:
content = read_pdf_to_string(path)

In [5]:
content

'\n2. Tích hợp Network Load Balancer (NLB) trên VKS\n2.1. Network Load Balancer (NLB) là gì?\nNetwork Load Balancer (tên viết tắt là NLB) là một bộ cân bằng tải được cung cấp bởi VNGCloud giúp phân phối lưu lượng truy cập\nmạng đến nhiều máy chủ back-end (backend servers) trong một nhóm máy tính (instance group). NLB hoạt động ở layer 4 của mô hình\nOSI, giúp cân bằng tải dựa trên địa chỉ IP và cổng TCP/UDP. Để biết thêm thông tin chi tiết về NLB, vui lòng tham khảo tại\nhttps://docs.vngcloud.vn/vng-cloud-document/vn/vks/network/lam-viec-voi-network-load-balancing-nlb\n2.2. Mô hình triển khai\n(Đường dẫn đến hình ảnh: https://docs.vngcloud.vn/~gitbook/image?url=https%3A%2F%2F3672463924-\nﬁles.gitbook.io%2F%7E%2Fﬁles%2Fv0%2Fb%2Fgitbook-x-\nprod.appspot.com%2Fo%2Fspaces%252FB0NrrrdJdpYOYzRkbWp5%252Fuploads%252FVYBtJjEoUNgDi1f5J9vL%252Fimage.png%3Falt%3\nDmedia%26token%3D554a2d62-320e-48d1-a884-3c7cce589071&width=768&dpr=2&quality=100&sign=4336a109&sv=2)\nVNGCloud LoadBalancer Controller:

Breakpoint types:
  - 'percentile': all differences between sentences are calculated, and then any difference greater than the X percentile is split.
  - 'standard_deviation': any difference greater than X standard deviations is split.
  - 'interquartile': the interquartile distance is used to split chunks.

In [6]:
text_splitter = SemanticChunker(HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2"), breakpoint_threshold_type='percentile', breakpoint_threshold_amount=90) # chose which embeddings and breakpoint type and threshold to use

/home/stackops/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Split original text to semantic chunks


In [7]:
docs = text_splitter.create_documents([content])


## Create vector store and retriever


In [8]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2")
vectorstore = FAISS.from_documents(docs, embeddings)
chunks_query_retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

## Test the retriever


In [9]:
test_query = "Làm sao để sử dụng lại một NLB"
context = retrieve_context_per_question(test_query, chunks_query_retriever)
show_context(context)

Context 1:
Dưới đây là một vài tình huống sử dụng NLB trên VKS:
Ví dụ 1: Công khai một dịch vụ Nginx trên cổng 80 với NLB đến Internet:
apiVersion: apps/v1
kind: Deployment
metadata:
  name: nginx-app
spec:
  selector:
    matchLabels:
      app: nginx
  replicas: 1 
  template:
    metadata:
      labels:
        app: nginx
    spec:
      containers:
      - name: nginx
        image: nginx:1.19.1
        ports:
        - containerPort: 8 0 
---
apiVersion: v1
kind: Service
metadata:
  name: nginx-service
spec:
  selector:
    app: nginx
  type: LoadBalancer 
  ports:
    - protocol: TCP
      port: 8 0 
      targetPort: 8 0 
Ví dụ 2: Triển khai dịch vụ Apache HTTP và chỉ cho phép truy cập nội bộ qua port 8080:
apiVersion: apps/v1
kind: Deployment
metadata:
  name: internal-http-apache2-deployment
spec:
  replicas: 2 
  selector:
    matchLabels:
      app: apache2
  template:
    metadata:
      labels:
        app: apache2
    spec:
      containers:
        - name: apache2
      